# Somatic Variants from KF-TALL were pulled directly from consensus.maf files, (NOT the VWB somatic tables, like the NBL data was)

### Like the NBL somatic variants, these variants have been filtered to include VEP IMPACT = HIGH.
More info can be found at the somatic_files.ipynb in the U24 Data Studio session in the Taylor_Urbs project on CAVATICA

In [2]:
#scp /Users/stearb/Downloads/TALL_somatic_variants_VEP_HIGH_July7th.csv \
#stearb-hpc9.research.chop.edu:/home/stearb/U24/data/somatic_variants/

In [3]:
import pandas as pd
import polars as pl
import numpy as np

def fill_missing_cols(df):
    if 'node_id' not in df.columns:
        raise ValueError('Must have at least a "node_id" column.')
    
    all_cols = set([ 'node_label', 'node_synonyms', 'node_dbxrefs',
            'node_definition','node_namespace','value','lowerbound','upperbound','unit'])
    missing_cols = list(all_cols - set(df.columns))
    nan_cols_df = pd.DataFrame(np.full([len(df), len(missing_cols)], np.nan),columns=missing_cols)

    if isinstance(df, pd.DataFrame):
        nan_cols_df.index = df.index
        return pd.concat([df,nan_cols_df],axis=1)
    elif isinstance(df, pl.DataFrame):
        return pl.concat([df,pl.from_pandas(nan_cols_df)],how='horizontal') # no index for polars
    else:
        raise ValueError(f'Must Pass either a pandas DataFrame or a polars DataFrame but recieved "{type(df)}".')
    

def bin_column(df, col2bin=''):
    '''works with polars df'''
    
    '''tpm_bins = list([0.0000000,7e-4,8e-4,9e-4]) + list(np.linspace(1e-3,9e-3,9)) + \
               list(np.round(np.linspace(1e-2,9e-2,9),2)) + list(np.round(np.linspace(.1,1,10),2)) + \
               list(np.linspace(2,100,99)) + list(np.arange(100,1100,100)[1:]) +  \
                list(np.arange(2000,11000,1000)) + list(np.arange(20_000,110_000,10_000)) + [300_000] '''
    
    bins = np.unique(hscloChrom['lowerbound'].values) #hscloChrom['lowerbound'].values
    # check that none of the disease expression values are out of range, ie larger than the max bin value
    # use for pandas
    #if np.sum(df['Disease_MeanTpm'] > np.max(tpm_bins)) == 0:
        
    # use for polars
    if len(df.filter(df[col2bin] > np.max(bins))) == 0:
        df_binned = df.with_columns(  df[col2bin].cut(breaks=bins).alias('bins')  )
        # check that order has been maintained #assert pl.all(df[col2bin] == df_binned[col2bin])
    else:
        print('OUT OF RANGE ERROR!')
        assert False 
    return df_binned

In [9]:
df = pd.read_csv('/home/stearb/U24/data/somatic_variants/TALL_somatic_variants_VEP_HIGH_July7th.csv')
df.head()

,HGVSg,Hugo_Symbol,HGVSp,dbSNP_RS,Consequence,SYMBOL_SOURCE,HGNC_ID,BIOTYPE,IMPACT,gnomad_3_1_1_AF,gnomad_3_1_1_AF_popmax,gnomad_3_1_1_AF_non_cancer_popmax,gnomad_3_1_1_AF_non_cancer_all_popmax,Matched_Norm_Sample_Barcode,Tumor_Sample_Barcode,path
0,chr1:g.26697414G>A,ARID1A,p.Trp337Ter,novel,stop_gained,HGNC,HGNC:11110,protein_coding,HIGH,.,.,.,.,BS_XD2PEVH1,BS_190CS8A4,/sbgenomics/project-files/000ef953-f37a-411b-8...
1,chr6:g.41936013dup,CCND3,p.Arg271ProfsTer53,novel,frameshift_variant,HGNC,HGNC:1585,protein_coding,HIGH,6.5e-06,1.45e-05,1.5e-05,1.5e-05,BS_XD2PEVH1,BS_190CS8A4,/sbgenomics/project-files/000ef953-f37a-411b-8...
2,chr12:g.79594385G>A,PAWR,p.Gln294Ter,novel,stop_gained,HGNC,HGNC:8614,protein_coding,HIGH,.,.,.,.,BS_XD2PEVH1,BS_190CS8A4,/sbgenomics/project-files/000ef953-f37a-411b-8...
3,chr15:g.45417349C>T,SPATA5L1,p.Arg640Ter,rs199667349,stop_gained,HGNC,HGNC:28762,protein_coding,HIGH,8.5e-05,0.000132,0.000123,0.000123,BS_XD2PEVH1,BS_190CS8A4,/sbgenomics/project-files/000ef953-f37a-411b-8...
4,chr19:g.5923819del,RANBP3,p.Glu365ArgfsTer79,novel,frameshift_variant,HGNC,HGNC:9850,protein_coding,HIGH,.,.,.,.,BS_XD2PEVH1,BS_190CS8A4,/sbgenomics/project-files/000ef953-f37a-411b-8...


In [6]:
df[['HGVSg']].sample(12)

#[i[-3:] for i in df['HGVSg']]

,HGVSg
2506,chr10:g.73016878C>A
4539,chr1:g.186367984delinsGG
10596,chrM:m.15611G>A
1395,chr20:g.62310830C>T
7790,chr10:g.68292024C>A
8320,chr5:g.33576677C>A
3569,chr12:g.12718026_12718032del
7887,chr8:g.21985665_21985666insCGTTCACTCC
13865,chr12:g.56716428delinsGTG
993,chr1:g.92834790_92834794del


In [4]:
import collections
collections.Counter([i[-3:] for i in df['HGVSg']]) 'dup', 'del', 'inv', 

Counter({'del': 3237,
         'G>T': 2507,
         'C>A': 2384,
         'dup': 1395,
         'G>A': 1298,
         'C>T': 1273,
         'A>G': 162,
         'T>C': 141,
         'A>T': 139,
         'T>G': 116,
         'A>C': 95,
         'T>A': 92,
         'CCC': 71,
         'G>C': 60,
         'C>G': 58,
         'nsA': 54,
         'sGG': 52,
         'GGG': 51,
         'CCG': 47,
         'nsT': 44,
         'CTC': 39,
         'nsG': 39,
         'TCG': 39,
         'sCC': 36,
         'ACC': 33,
         'CCT': 32,
         'TCC': 32,
         'sTC': 29,
         'GGA': 24,
         'nsC': 24,
         'GGT': 24,
         'AAG': 22,
         'TCT': 22,
         'GGC': 21,
         'GAA': 21,
         'AGG': 21,
         'GTC': 21,
         'GCT': 20,
         'AAA': 20,
         'TTC': 19,
         'sGA': 19,
         'GCC': 19,
         'TTT': 18,
         'GAG': 18,
         'AGA': 18,
         'TGG': 18,
         'CTT': 17,
         'GTG': 17,
         'ATC': 16,
    

In [10]:
hgnc_master = pd.read_csv('/home/stearb/U24/data/helper_files/hgnc_master.txt',sep='\t')


/tmp/ipykernel_1628114/2102876145.py:1: DtypeWarning: Columns (32,34,38,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  hgnc_master = pd.read_csv('/home/stearb/U24/data/helper_files/hgnc_master.txt',sep='\t')


In [11]:
# merge in ensembl gene ids
df = pd.merge(df,hgnc_master[['hgnc_id','ensembl_gene_id']].rename({'hgnc_id':'HGNC_ID'},axis=1))

# Define codes
### Need to get mappings to ensembl proteins and ensembl transcripts

In [12]:
# variant
df['HGVSg'] = df['HGVSg'].replace({':': '.'},regex=True) 
df['hgvsg_variant_code'] = 'HGVSG:' + df['HGVSg'] 

# ensembl gene
df['ens_gene_code'] = 'ENSEMBL:' + df['ensembl_gene_id']

# disease/MONDO for 'T-cell childhood acute lymphocytic leukemia'
# https://www.ebi.ac.uk/ols4/search?q=t-all&ontology=mondo
df['disease_code'] = 'MONDO:0000871'

# cohort
df['cohort_code'] = 'KFCOHORT:SD-AQ9KVN5P'

# study
df['study_code'] = 'KFSTUDY:KF-TALL-SOMATIC'

# Define edges

### Variant - Gene

In [13]:
e_var_gene = df[['hgvsg_variant_code','ens_gene_code']].dropna().drop_duplicates().reset_index(drop=True)
e_var_gene.columns = ['subject','object']

e_var_gene['predicate'] = 'related_to_gene'
e_var_gene = e_var_gene[['subject','predicate','object']]

e_var_gene.sample(5)

,subject,predicate,object
5407,HGVSG:chr15.g.23645455G>T,related_to_gene,ENSEMBL:ENSG00000254585
12710,HGVSG:chr3.g.195864183A>C,related_to_gene,ENSEMBL:ENSG00000061938
2361,HGVSG:chr17.g.66223697C>T,related_to_gene,ENSEMBL:ENSG00000091583
5116,HGVSG:chr16.g.8894828G>A,related_to_gene,ENSEMBL:ENSG00000187555
4175,HGVSG:chr1.g.244705700G>T,related_to_gene,ENSEMBL:ENSG00000121644


In [18]:
pd.set_option('display.max_columns', None)
df.head(2)

,HGVSg,Hugo_Symbol,HGVSp,dbSNP_RS,Consequence,SYMBOL_SOURCE,HGNC_ID,BIOTYPE,IMPACT,gnomad_3_1_1_AF,gnomad_3_1_1_AF_popmax,gnomad_3_1_1_AF_non_cancer_popmax,gnomad_3_1_1_AF_non_cancer_all_popmax,Matched_Norm_Sample_Barcode,Tumor_Sample_Barcode,path,ensembl_gene_id,hgvsg_variant_code,ens_gene_code,disease_code,cohort_code,study_code
0,chr1.g.26697414G>A,ARID1A,p.Trp337Ter,novel,stop_gained,HGNC,HGNC:11110,protein_coding,HIGH,.,.,.,.,BS_XD2PEVH1,BS_190CS8A4,/sbgenomics/project-files/000ef953-f37a-411b-8...,ENSG00000117713,HGVSG:chr1.g.26697414G>A,ENSEMBL:ENSG00000117713,MONDO:0000871,KFCOHORT:SD-AQ9KVN5P,KFSTUDY:KF-TALL-SOMATIC
1,chr6.g.41936013dup,CCND3,p.Arg271ProfsTer53,novel,frameshift_variant,HGNC,HGNC:1585,protein_coding,HIGH,6.5e-06,1.45e-05,1.5e-05,1.5e-05,BS_XD2PEVH1,BS_190CS8A4,/sbgenomics/project-files/000ef953-f37a-411b-8...,ENSG00000112576,HGVSG:chr6.g.41936013dup,ENSEMBL:ENSG00000112576,MONDO:0000871,KFCOHORT:SD-AQ9KVN5P,KFSTUDY:KF-TALL-SOMATIC


### Variant - Protein   --- *****No ENSEMBL protein IDs for the TALL dataset****

In [17]:

# Some extra formatting needs to be done for the protein code...

# Drop rows where its NaN
df_prot = df.dropna(subset=['hgvsp'])

# DROP rows where the ENSP is not defined.
df_prot = df_prot[df_prot['hgvsp'].str.startswith('ENSP')]

df_prot['ens_protein_code'] = 'ENSEMBL:' + pd.Series([i.split('.')[0] for i in df_prot['hgvsp']])

e_var_prot = df_prot[['hgvsg_variant_code','ens_protein_code']]
e_var_prot.columns = ['subject','object']
e_var_prot['predicate'] = 'has_protein'
e_var_prot = e_var_prot[['subject','predicate','object']].dropna().drop_duplicates().reset_index(drop=True)
e_var_prot.sample(5)

KeyError: ['hgvsp']

In [14]:

# Drop rows where its NaN
df_prot = df.dropna(subset=['HGVSp'])

# DROP rows where the ENSP is not defined.
df_prot = df_prot[df_prot['hgvsp'].str.startswith('ENSP')]

df_prot['ens_protein_code'] = 'ENSEMBL:' + pd.Series([i.split('.')[0] for i in df_prot['hgvsp']])


KeyError: ['hgvsp']

In [ ]:
# Drop rows where its NaN
df_prot = df.dropna(subset=['hgvsp'])

# DROP rows where the ENSP is not defined.
df_prot = df_prot[df_prot['hgvsp'].str.startswith('ENSP')]

df_prot['ens_protein_code'] = 'ENSEMBL:' + pd.Series([i.split('.')[0] for i in df_prot['hgvsp']])


### Variant - Cohort

In [8]:
e_var_cohort = df[['hgvsg_variant_code','cohort_code']].dropna().drop_duplicates().reset_index(drop=True)
e_var_cohort.columns = ['subject','object']
e_var_cohort['predicate'] = 'belongs_to_cohort'
e_var_cohort = e_var_cohort[['subject','predicate','object']]
e_var_cohort.sample(5)

,subject,predicate,object
248,HGVSG:chr20.g.3044514G>T,belongs_to_cohort,KFCOHORT:SD-AQ9KVN5P
13176,HGVSG:chr20.g.50903900C>A,belongs_to_cohort,KFCOHORT:SD-AQ9KVN5P
9464,HGVSG:chr14.g.26448240T>A,belongs_to_cohort,KFCOHORT:SD-AQ9KVN5P
11522,HGVSG:chr11.g.60339865G>T,belongs_to_cohort,KFCOHORT:SD-AQ9KVN5P
12570,HGVSG:chr12.g.112887834G>T,belongs_to_cohort,KFCOHORT:SD-AQ9KVN5P


### Study - Cohort

In [9]:
e_study_cohort = df[['study_code','cohort_code']].dropna().drop_duplicates().reset_index(drop=True)
e_study_cohort.columns = ['subject','object']

e_study_cohort['predicate'] = 'study_has_cohort'
e_study_cohort = e_study_cohort[['subject','predicate','object']]
e_study_cohort

,subject,predicate,object
0,KFSTUDY:KF-TALL-SOMATIC,study_has_cohort,KFCOHORT:SD-AQ9KVN5P


### Variant - HSCLO

In [10]:
q = pl.scan_csv('/home/stearb/U24/data/HSCLO/OWLNETS_edgelist_HSCLO.tsv',separator='\t')
hsclo = q.select(['node_id','lowerbound','upperbound']).collect()
hsclo = hsclo.to_pandas()

chroms = np.unique([i.split(' ')[-1].split('.')[0] for i in hsclo.node_id.astype(str)])
chroms = [i.tolist() for i in chroms]  
chroms.remove('Human_Genome_hg38')
chroms.remove('MtDNA')
chroms

['chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr20',
 'chr21',
 'chr22',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chrX',
 'chrY']

In [11]:
e_var_hsclo_MASTER = list()

# Go by chromosome and merge each variant to the correct HSCLO term/code
# most of this is extracting the variant position from the hgvsg_variant_code col
# and saving it to the variant_position col. Then bin on this col.

for CHROM in chroms:
    
    print(CHROM)
    
    # search for 'chr1.' or 'chr2.' if CHROM is either 1 or 2. This prevents matching on 11,12,13, when matching on 'chr1'
    if CHROM == 'chr1': CHROM = 'chr1\.'
    elif CHROM == 'chr2': CHROM == 'chr2\.'
    
    # Select variant col which has the variant chromosomal position for just current CHROM
    dfChrom = df[df['hgvsg_variant_code'].str.contains(CHROM)][['hgvsg_variant_code']].dropna()
    
    # format to extract just numerical position 
    dfChrom['variant_position'] = \
          [i.split('.')[-1].replace('del','').replace('ins','')\
            .replace('A','').replace('C','').replace('T','').replace('G','')\
            .replace('>','') if 'inv' not in i or 'dup' not in i else i for i\
            in dfChrom['hgvsg_variant_code']]

    dfChrom['variant_position'] = dfChrom['variant_position'].replace('dup','',regex=True).replace('inv','',regex=True)
    dfChrom['variant_position'] = [i.split('_')[0] for i in dfChrom['variant_position']]
    dfChrom['variant_position'] = dfChrom['variant_position'].astype(int)
    dfChrom = dfChrom.drop_duplicates().reset_index(drop=True)
    
    # get hsclo codes for current CHROM
    hscloChrom = hsclo[hsclo['node_id'].str.contains(CHROM)].dropna()

    ###### do the binning
    dfChromBins = bin_column(pl.DataFrame(dfChrom),col2bin='variant_position').to_pandas()

    # make lowerbound col to merge in hsclo codes
    dfChromBins['lowerbound'] = [float(i.split(',')[0][1:]) for i in dfChromBins['bins']]

    # merge in hsclo codes 
    dfMerged = pd.merge(dfChromBins,hscloChrom,on='lowerbound',how='left')
    
    # this may introduce an error, if im matching on lowerbound only, as there are multiple resolution
    # levels, so upperbound must also be specified!
    # ...check that the variant_position is b/t lower and upperbound
    print(len(dfMerged) ,len(dfMerged[(dfMerged['variant_position'] > dfMerged['lowerbound']) &\
        (dfMerged['variant_position'] < dfMerged['upperbound'])]))
    
    #assert dfMerged.shape == dfMerged[(dfMerged['variant_position'] > dfMerged['lowerbound']) &\
    #    (dfMerged['variant_position'] < dfMerged['upperbound'])].shape
    
    e_var_hsclo = dfMerged[['hgvsg_variant_code','node_id']].drop_duplicates().dropna().reset_index(drop=True)
    e_var_hsclo.columns = ['subject','object']
    e_var_hsclo['predicate'] = 'has_location'
    e_var_hsclo = e_var_hsclo[['subject','predicate','object']]
    
    # e_var_hsclo is longer than dfChrom bc each variant is mapped to multiple HSCLO resolution levels
    #print(len(e_var_hsclo),len(dfChrom)) 
    
    if len(e_var_hsclo_MASTER) == 0:
        e_var_hsclo_MASTER = e_var_hsclo
    else:
        e_var_hsclo_MASTER = pd.concat([e_var_hsclo_MASTER,e_var_hsclo])
       
e_var_hsclo_MASTER = e_var_hsclo_MASTER.drop_duplicates().dropna().reset_index(drop=True)


chr1
1282 1279
chr10
730 728
chr11
902 899
chr12
774 769
chr13
246 244
chr14
563 562
chr15
442 440
chr16
706 704
chr17
764 760
chr18
181 181
chr19
879 879
chr2
4459 4443
chr20
371 368
chr21
141 141
chr22
261 260
chr3
727 726
chr4
619 616
chr5
721 718
chr6
675 674
chr7
610 607
chr8
457 457
chr9
807 807
chrX
668 667
chrY
11 11


In [22]:
e_var_hsclo_MASTER

,subject,predicate,object
0,HGVSG:chr1.g.26697414G>A,has_location,HSCLO chr1.26697001-26698000
1,HGVSG:chr1.g.33586525C>A,has_location,HSCLO chr1.33586001-33587000
2,HGVSG:chr1.g.24652520del,has_location,HSCLO chr1.24652001-24653000
3,HGVSG:chr1.g.167128564C>T,has_location,HSCLO chr1.167128001-167129000
4,HGVSG:chr1.g.228279243G>T,has_location,HSCLO chr1.228279001-228280000
...,...,...,...
17218,HGVSG:chrY.g.12720654del,has_location,HSCLO chrY.12720001-12721000
17219,HGVSG:chrY.g.13251168del,has_location,HSCLO chrY.13251001-13252000
17220,HGVSG:chrY.g.19739546del,has_location,HSCLO chrY.19739001-19740000
17221,HGVSG:chrY.g.2787297C>A,has_location,HSCLO chrY.2787001-2788000


# Save edges

In [12]:
edges_all = pd.concat([e_var_gene,e_var_cohort,e_study_cohort,
                       e_var_hsclo_MASTER])   # e_var_prot,e_var_trans, e_trans_prot

edges_all = edges_all.drop_duplicates().dropna().reset_index(drop=True)

edges_all.to_csv('/mnt/isilon/opentargets/U24KG/data/owlnets_nodes_and_edges/TALL_somatic/OWLNETS_edgelist.txt',
             sep= "\t",index=False)

In [15]:
nodes_all

,node_id,node_label,node_definition,node_synonyms,upperbound,node_namespace,unit,value,node_dbxrefs,lowerbound
0,HGVSG:chr1.g.26697414G>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HGVSG:chr6.g.41936013dup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HGVSG:chr12.g.79594385G>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HGVSG:chr15.g.45417349C>T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HGVSG:chr19.g.5923819del,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20545,ENSEMBL:ENSG00000105298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20546,ENSEMBL:ENSG00000184635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20547,ENSEMBL:ENSG00000163159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20548,ENSEMBL:ENSG00000113068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save nodes

In [13]:
nodes_all = pd.concat([edges_all['subject'],edges_all['object']])\
                          .drop_duplicates().dropna().reset_index(drop=True)

nodes_all = pd.DataFrame(nodes_all,columns=['node_id'])
nodes_all = fill_missing_cols(nodes_all)

# NO Need to save HSCLO node ids
nodes_all = nodes_all[~nodes_all['node_id'].str.startswith('HSCLO')]\
    .drop_duplicates().dropna(subset=['node_id']).reset_index(drop=True)

nodes_all

,node_id,node_namespace,node_definition,unit,node_label,node_dbxrefs,lowerbound,value,node_synonyms,upperbound
0,HGVSG:chr1.g.26697414G>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HGVSG:chr6.g.41936013dup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HGVSG:chr12.g.79594385G>A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HGVSG:chr15.g.45417349C>T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HGVSG:chr19.g.5923819del,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20545,ENSEMBL:ENSG00000105298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20546,ENSEMBL:ENSG00000184635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20547,ENSEMBL:ENSG00000163159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20548,ENSEMBL:ENSG00000113068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:

nodes_all.to_csv(
    '/mnt/isilon/opentargets/U24KG/data/owlnets_nodes_and_edges/TALL_somatic/OWLNETS_node_metadata.txt',
             sep= "\t",index=False)